In [3]:
%reload_ext autoreload
%autoreload 2
from pathlib import Path
from sklearn.model_selection import train_test_split

import sys
sys.path.append("..")
from dataset import ManualFeatureDataset, ManualFeatureDataModule

## Train/Test Split

In [4]:
root_dir = Path("/media/nvme1/icare-data/6h-features")
labels_csv = Path("/home/bc299/icare/patient_data.csv")

## Dataset Setup

In [5]:
all_patient_ids = [dir_.name for dir_ in root_dir.iterdir()]
train_ids, temp_ids = train_test_split(all_patient_ids, test_size=0.3, random_state=42)
val_ids, test_ids = train_test_split(temp_ids, test_size=2/3, random_state=42)

In [6]:
dataset = ManualFeatureDataset(root_dir, labels_csv, train_ids)

In [7]:
data_module = ManualFeatureDataModule(root_dir=root_dir,
                                      labels_csv=labels_csv,
                                      batch_size=32)
data_module.setup()

In [8]:
for batch_idx, (x, y) in enumerate(data_module.test_dataloader()):
    print(x.size())

torch.Size([32, 8, 144])
torch.Size([32, 8, 144])
torch.Size([32, 8, 144])
torch.Size([32, 8, 144])
torch.Size([32, 8, 144])
torch.Size([32, 8, 144])
torch.Size([32, 8, 144])
torch.Size([32, 8, 144])
torch.Size([32, 8, 144])
torch.Size([32, 8, 144])
torch.Size([32, 8, 144])
torch.Size([32, 8, 144])
torch.Size([32, 8, 144])
torch.Size([32, 8, 144])
torch.Size([32, 8, 144])
torch.Size([32, 8, 144])
torch.Size([32, 8, 144])
torch.Size([32, 8, 144])
torch.Size([32, 8, 144])
torch.Size([32, 8, 144])
torch.Size([32, 8, 144])
torch.Size([32, 8, 144])
torch.Size([32, 8, 144])
torch.Size([32, 8, 144])
torch.Size([32, 8, 144])
torch.Size([32, 8, 144])
torch.Size([32, 8, 144])
torch.Size([32, 8, 144])
torch.Size([32, 8, 144])
torch.Size([32, 8, 144])
torch.Size([32, 8, 144])
torch.Size([32, 8, 144])
torch.Size([32, 8, 144])
torch.Size([32, 8, 144])
torch.Size([32, 8, 144])
torch.Size([2, 8, 144])


## Train Model

In [20]:
import torch
import lightning.pytorch as pl
from torch.optim import Adam
from torch.nn.functional import nll_loss
from model import BiLSTMClassifier

In [21]:
class BiLSTMClassifierModule(pl.LightningModule):
    def __init__(self, input_size, hidden_size, num_layers, dropout, learning_rate):
        super().__init__()

        self.model = BiLSTMClassifier(input_size, hidden_size, num_layers, dropout)
        self.learning_rate = learning_rate

    def forward(self, x):
        return self.model(x)

    def training_step(self, batch, batch_idx):
        x, y = batch
        logits = self.model(x)
        loss = nll_loss(logits, y)
        preds = torch.argmax(logits, dim=1)
        acc = (preds == y).float().mean()
        self.log("train_loss", loss)
        self.log("train_acc", acc)
        return loss
    
    def validation_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = nll_loss(logits, y)
        preds = torch.argmax(logits, dim=1)
        acc = (preds == y).float().mean()
        self.log("val_loss", loss, prog_bar=True)
        self.log("val_acc", acc, prog_bar=True)

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=self.learning_rate, weight_decay=0.01)
        return optimizer

In [25]:
from lightning.pytorch.loggers import TensorBoardLogger
from lightning.pytorch import Trainer

data_module = ManualFeatureDataModule(root_dir, labels_csv, batch_size=32)
model = BiLSTMClassifierModule(input_size=8, hidden_size=128, num_layers=4, dropout=0.5, learning_rate=1e-5)
logger = TensorBoardLogger("tb_logs", name="test_classifier")
trainer = Trainer(max_epochs=50, logger=logger)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [26]:
trainer.fit(model, data_module)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type             | Params
-------------------------------------------
0 | model | BiLSTMClassifier | 1.3 M 
-------------------------------------------
1.3 M     Trainable params
0         Non-trainable params
1.3 M     Total params
5.310     Total estimated model params size (MB)


/home/bc299/miniconda3/envs/icare/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:442: PossibleUserWarning: The dataloader, val_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 32 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/home/bc299/miniconda3/envs/icare/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:442: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 32 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Epoch 49: 100%|██████████| 123/123 [00:00<00:00, 171.26it/s, v_num=11, val_loss=0.665, val_acc=0.627]

`Trainer.fit` stopped: `max_epochs=50` reached.


Epoch 49: 100%|██████████| 123/123 [00:00<00:00, 167.84it/s, v_num=11, val_loss=0.665, val_acc=0.627]
